Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Anjana Tissera"
ID = "st123459"

---

# Lab 06: Generative classifiers: Naive Bayes

As discussed in class, a Naive Bayes classifier works as follows:
$$\begin{eqnarray}
p(y \mid \mathbf{x} ; \theta) & = & \frac{p(\mathbf{x} \mid y ; \theta) p(y ; \theta)}{p(\mathbf{x} ; \theta)} \\
& \propto & p(\mathbf{x} \mid y ; \theta) p(y ; \theta) \\
& \approx & p(y ; \theta) \prod_j p(x_j \mid y ; \theta)
\end{eqnarray}$$
We will use Naive Bayes to perform diabetes diagnosis and text classification.

## Example 1: Diabetes classification

In this example we predict wheter a patient with specific diagnostic measurements has diabetes or not. As the features are
continuous, we will model the conditional probabilities
$p(x_j \mid y ; \theta)$ as univariate Gaussians with mean $\mu_{j,y}$ and standard deviation $\sigma_{j,y}$.

The data are originally from the U.S. National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) and are available
from [Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database)


In [2]:
import csv
import math
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Data manipulation

First we have some functions to read the dataset, split it into train and test, and partition it according to target class ($y$).

In [3]:
# Load data from CSV file
def loadCsv(filename):
    data_raw = pd.read_csv(filename)
    headers = data_raw.columns
    dataset = data_raw.values
    return dataset, headers

# Split dataset into test and train with given ratio
def splitDataset(test_size,*arrays,**kwargs):
    return train_test_split(*arrays,test_size=test_size,**kwargs)

# Separate training data according to target class
# Return key value pairs array in which keys are possible target variable values
# and values are the data records.

def data_split_byClass(dataset):
    Xy = {}
    for i in range(len(dataset)):
        datapair = dataset[i]
        # datapair[-1] (the last column) is the target class for this record.
        # Check if we already have this value as a key in the return array
        if (datapair[-1] not in Xy):
            # Add class as key
            Xy[datapair[-1]] = []
        # Append this record to array of records for this class key
        Xy[datapair[-1]].append(datapair)
    return Xy

### Model training

Next we have some functions used for training the model. Parameters include mean and standard deviation, used
to partition numerical variables into categorical variables, as well as 

In [4]:
# Parameters of a Gaussian are its mean and standard deviation

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

# Calculate Gaussian parameters mu and sigma for each attribute over a dataset

def get_gaussian_parameters(X,y):
    parameters = {}
    unique_y = np.unique(y)
    for uy in unique_y:
        mean = np.mean(X[y==uy],axis=0)
        std = np.std(X[y==uy],axis=0)
        py = y[y==uy].size/y.size
        parameters[uy] = {'prior':py,'mean':mean,'std':std}
    return parameters, unique_y

def calculateProbability(x, mu, sigma):
    sigma = np.diag(sigma**2)
    x = x.reshape(-1,1)
    mu = mu.reshape(-1,1)
    exponent = np.exp(-1/2*(x-mu).T@np.linalg.inv(sigma)@(x-mu))
    return ((1/(np.sqrt(((2*np.pi)**x.size)*np.linalg.det(sigma))))*exponent)[0,0]

### Model testing

Next some functions for testing the model on a test set and computing its accuracy. Note that we assume
$$ p(y \mid \mathbf{x} ; \theta) \propto p(\mathbf{x} \mid y ; \theta), $$
which means we assume that the priors $p(y)$ are equal for each possible value of $y$.

In [5]:
# Calculate class conditional probabilities for given input data vector

def predict_one(x,parameters,unique_y,prior = True):
    probabilities = []
    for key in parameters.keys():
        probabilities.append(calculateProbability(x,parameters[key]['mean'],parameters[key]['std'])*(parameters[key]['prior']**(float(prior))))
    probabilities = np.array(probabilities)
    return unique_y[np.argmax(probabilities)]

def getPredictions(X, parameters, unique_y,prior=True):
    predictions = []
    for i in range(X.shape[0]):
        predictions.append(predict_one(X[i],parameters,unique_y,prior))
    return np.array(predictions)

# Get accuracy for test set

def getAccuracy(y, y_pred):
    correct = len(y[y==y_pred])
    return correct/y.size

### Experiment

Here we load the diabetes dataset, split it into training and test data, train a Gaussian NB model, and test the model on the test set.

In [6]:
# Load dataset

filename = 'diabetes.csv'
dataset, headers = loadCsv(filename)
#print(headers)
#print(np.array(dataset)[0:5,:])

# Split into training and test

X_train,X_test,y_train,y_test = splitDataset(0.4,dataset[:,:-1],dataset[:,-1])
print("Total =",len(dataset),"Train =", len(X_train),"Test =",len(X_test))

# Train model

parameters, unique_y = get_gaussian_parameters(X_train,y_train)
prediction = getPredictions(X_test,parameters,unique_y)
print("Accuracy with Prior =",getAccuracy(y_test,prediction))

# Test model

prediction = getPredictions(X_test,parameters,unique_y,prior = False)
print("Accuracy without Prior =",getAccuracy(y_test,prediction))

Total = 768 Train = 460 Test = 308
Accuracy with Prior = 0.7337662337662337
Accuracy without Prior = 0.7272727272727273


###  Exercise In lab / take home work (20 points)

Find out the proportion of the records in your dataset are positive vs. negative.  Can we conclude that $p(y=1) = p(y=0)$? If not, add
the priors $p(y=1)$ and $p(y=0)$ to your NB model. Does it improve the result?



In [7]:
# YOUR CODE HERE

#Proportion of the records in the dataset that are positive vs. negative
y_train_int = (y_train).astype(int)
m_train = y_train_int.shape[0]
prior_y_1 = np.count_nonzero(y_train_int == 1) / m_train
prior_y_0 = np.count_nonzero(y_train_int == 0) / m_train 

print("Proportion of positive records: ", prior_y_1 )
print("Proportion of negative records: ", prior_y_0 )

Proportion of positive records:  0.3521739130434783
Proportion of negative records:  0.6478260869565218


**Explain that you can conclude that $p(y=1) = p(y=0)$? If not, add
the priors $p(y=1)$ and $p(y=0)$ to your NB model. Does it improve the result? (double click to explain)**



### Answer:

$p(y=1) = 0.3695652173913043$

$p(y=0) = 0.6304347826086957$

Therefore we cannot conclude that $p(y=1) = p(y=0)$



In [8]:
# Adding the priors  𝑝(𝑦=1)  and  𝑝(𝑦=0)  to NB model.

class GaussianNB:
    def __init__(self, include_prior=True):
        self.include_prior = include_prior
        self.n_samples = 0
        self.n_features = 0
        self.n_classes = 0
        self.mean = None
        self.variance = None
        self.priors = None
        
    def fit(self, X, y):
        self.n_samples, self.n_features = X.shape
        self.n_classes = len(np.unique(y))

        self.mean = np.zeros((self.n_classes, self.n_features))
        self.variance = np.zeros((self.n_classes, self.n_features))
        self.priors = np.zeros(self.n_classes)

        for c in range(self.n_classes):
            X_c = X[y == c]

            self.mean[c, :] = np.mean(X_c, axis=0)
            self.variance[c, :] = np.var(X_c, axis=0)
            self.priors[c] = X_c.shape[0] / self.n_samples
            
    def predict(self, X):
        y_hat = [self._get_class_probability(x) for x in X]
        return np.array(y_hat)
    
    def _get_class_probability(self, x):
        posteriors = list()
        for c in range(self.n_classes):
            mean = self.mean[c]
            variance = self.variance[c]
            prior = self.priors[c]
            likelihood = self._gaussian_density(x, mean, variance)
            total_likelihood = np.prod(likelihood)
            
            if self.include_prior:
                posterior = prior * total_likelihood
            else:
                posterior = total_likelihood
            
            posteriors.append(posterior)
        
        return np.argmax(posteriors)    
    
    def _gaussian_density(self, x, mean, var):
        const = 1 / np.sqrt(var * 2 * np.pi)
        proba = np.exp(-0.5 * ((x - mean) ** 2 / var))
        return const * proba   
    
def get_gaussiannb_accuracy(y_true, y_hat):
    return np.sum(y_true==y_hat) / len(y_true)

In [9]:
df = pd.read_csv('diabetes.csv')

In [10]:
# Setting X and y
X = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
        'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]

y = df['Outcome']

# Train, test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=999)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(460, 8) (308, 8) (460,) (308,)


In [11]:
# Standardize X_train, X_test
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [12]:
# Model without including prior
gnb_no_prior = GaussianNB(include_prior=False)
gnb_no_prior.fit(X_train, y_train)
yhat_no_prior = gnb_no_prior.predict(X_test)
print('Naive Bayes accuracy without including priors: ', get_gaussiannb_accuracy(y_test, yhat_no_prior))

Naive Bayes accuracy without including priors:  0.7467532467532467


In [13]:
# Model without including prior
gnb_with_prior = GaussianNB(include_prior=True)
gnb_with_prior.fit(X_train, y_train)
yhat_with_prior = gnb_with_prior.predict(X_test)
print('Naive Bayes accuracy with including priors: ', get_gaussiannb_accuracy(y_test, yhat_with_prior))

Naive Bayes accuracy with including priors:  0.7564935064935064


### Discussion

Comparison of Gaussian Naive Bayes accuracy with and without priors include:

|  | With Priors | Without Priors |
| --- | --- | --- |
| Accuracy | 0.7564935064935064 | 0.7467532467532467 |


- From the above table we can see that when prior probabilities are inclded there is a small increase in the accuracy of the model.

## Example 2: Text classification

This example has been adapted from a post by Jaya Aiyappan, available at
[Analytics Vidhya](https://medium.com/analytics-vidhya/naive-bayes-classifier-for-text-classification-556fabaf252b#:~:text=The%20Naive%20Bayes%20classifier%20is,time%20and%20less%20training%20data).

We will generate a small dataset of sentences that are classified as either "statements" or "questions."

We will assume that occurance and placement of words within a sentence is independent of each other
(i.e., the features are conditionally independent given $y$). So the sentence "this is my book" is the same as "is this my book."
We will treat words as case insensitive.

In [14]:
# Generate text data for two classes, "statement" and "question"

text_train = [['This is my novel book', 'statement'],
              ['this book has more than one author', 'statement'],
              ['is this my book', 'question'],
              ['They are novels', 'statement'],
              ['have you read this book', 'question'],
              ['who is the novels author', 'question'],
              ['what are the characters', 'question'],
              ['This is how I bought the book', 'statement'],
              ['I like fictional characters', 'statement'],
              ['what is your favorite book', 'question']]

text_test = [['this is the book', 'statement'], 
             ['who are the novels characters', 'question'], 
             ['is this the author', 'question'],
            ['I like apples']]

# Load training and test data into pandas data frames

training_data = pd.DataFrame(text_train, columns= ['sentence', 'class'])
print(training_data)
print('\n------------------------------------------\n')
testing_data = pd.DataFrame(text_test, columns= ['sentence', 'class'])
print(testing_data)


                             sentence      class
0               This is my novel book  statement
1  this book has more than one author  statement
2                     is this my book   question
3                     They are novels  statement
4             have you read this book   question
5            who is the novels author   question
6             what are the characters   question
7       This is how I bought the book  statement
8         I like fictional characters  statement
9          what is your favorite book   question

------------------------------------------

                        sentence      class
0               this is the book  statement
1  who are the novels characters   question
2             is this the author   question
3                  I like apples       None


In [15]:
# Partition training data by class

stmt_docs = [train['sentence'] for index,train in training_data.iterrows() if train['class'] == 'statement']
question_docs = [train['sentence'] for index,train in training_data.iterrows() if train['class'] == 'question']
all_docs = [train['sentence'] for index,train in training_data.iterrows()]

# Get word frequencies for each sentence and class

def get_words(text):
    # Initialize word list
    words = [];
    # Loop through each sentence in input array
    for text_row in text: 
        # Check the number of words. Assume each word is separated by a blank space
        # so that the number of words is the number of blank spaces + 1
        number_of_spaces = text_row.count(' ')
        # loop through the sentence and get words between blank spaces.
        for i in range(number_of_spaces):
            # Check for for last word
            words.append([text_row[:text_row.index(' ')].lower()])
            text_row = text_row[text_row.index(' ')+1:]  
            i = i + 1        
        words.append([text_row])
    return np.unique(words)

# Get frequency of each word in each document

def get_doc_word_frequency(words, text):  
    word_freq_table = np.zeros((len(text),len(words)), dtype=int)
    i = 0
    for text_row in text:
        # Insert extra space between each pair of words to prevent
        # partial match of words
        text_row_temp = ''
        for idx, val in enumerate(text_row):
            if val == ' ':
                 text_row_temp = text_row_temp + '  '
            else:
                  text_row_temp = text_row_temp + val.lower()
        text_row = ' ' + text_row_temp + ' '
        j = 0
        for word in words: 
            word = ' ' + word + ' '
            freq = text_row.count(word)
            word_freq_table[i,j] = freq
            j = j + 1
        i = i + 1
    
    return word_freq_table

In [16]:
# Get word frequencies for statement documents

word_list_s = get_words(stmt_docs)
word_freq_table_s = get_doc_word_frequency(word_list_s, stmt_docs)
tdm_s = pd.DataFrame(word_freq_table_s, columns=word_list_s)
print(tdm_s)

   are  author  book  bought  characters  fictional  has  how  i  is  like  \
0    0       0     1       0           0          0    0    0  0   1     0   
1    0       1     1       0           0          0    1    0  0   0     0   
2    1       0     0       0           0          0    0    0  0   0     0   
3    0       0     1       1           0          0    0    1  1   1     0   
4    0       0     0       0           1          1    0    0  1   0     1   

   more  my  novel  novels  one  than  the  they  this  
0     0   1      1       0    0     0    0     0     1  
1     1   0      0       0    1     1    0     0     1  
2     0   0      0       1    0     0    0     1     0  
3     0   0      0       0    0     0    1     0     1  
4     0   0      0       0    0     0    0     0     0  


In [17]:
# Get word frequencies over all statement documents

freq_list_s = word_freq_table_s.sum(axis=0) 
freq_s = dict(zip(word_list_s,freq_list_s))
print(freq_s)

{'are': 1, 'author': 1, 'book': 3, 'bought': 1, 'characters': 1, 'fictional': 1, 'has': 1, 'how': 1, 'i': 2, 'is': 2, 'like': 1, 'more': 1, 'my': 1, 'novel': 1, 'novels': 1, 'one': 1, 'than': 1, 'the': 1, 'they': 1, 'this': 3}


In [18]:
# Get word frequencies for question documents

word_list_q = get_words(question_docs)
word_freq_table_q = get_doc_word_frequency(word_list_q, question_docs)
tdm_q = pd.DataFrame(word_freq_table_q, columns=word_list_q)
print(tdm_q)

   are  author  book  characters  favorite  have  is  my  novels  read  the  \
0    0       0     1           0         0     0   1   1       0     0    0   
1    0       0     1           0         0     1   0   0       0     1    0   
2    0       1     0           0         0     0   1   0       1     0    1   
3    1       0     0           1         0     0   0   0       0     0    1   
4    0       0     1           0         1     0   1   0       0     0    0   

   this  what  who  you  your  
0     1     0    0    0     0  
1     1     0    0    1     0  
2     0     0    1    0     0  
3     0     1    0    0     0  
4     0     1    0    0     1  


In [19]:
# Get word frequencies over all question documents

freq_list_q = word_freq_table_q.sum(axis=0) 
freq_q = dict(zip(word_list_q,freq_list_q))
print(freq_q)
print(freq_list_s)
print(freq_list_q)

{'are': 1, 'author': 1, 'book': 3, 'characters': 1, 'favorite': 1, 'have': 1, 'is': 3, 'my': 1, 'novels': 1, 'read': 1, 'the': 2, 'this': 2, 'what': 2, 'who': 1, 'you': 1, 'your': 1}
[1 1 3 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 3]
[1 1 3 1 1 1 3 1 1 1 2 2 2 1 1 1]


In [20]:
# Get word probabilities for statement class
a = 1
prob_s = []
for count in freq_list_s:
    #print(word, count)
    prob_s.append((count+a)/(sum(freq_list_s)+len(freq_list_s)*a))
#prob_s.append(a/(sum(freq_list_s)+len(freq_list_s)*a))
    
# Get word probabilities for question class

prob_q = []
for count in freq_list_q:
    prob_q.append((count+a)/(sum(freq_list_q)+len(freq_list_q)*a))
#prob_q.append(a/(sum(freq_list_q)+len(freq_list_q)*a))   
    
    
print('Probability of words for "statement" class \n')
print(dict(zip(word_list_s, prob_s)))
print('------------------------------------------- \n')
print('Probability of words for "question" class \n')
print(dict(zip(word_list_q, prob_q)))

Probability of words for "statement" class 

{'are': 0.043478260869565216, 'author': 0.043478260869565216, 'book': 0.08695652173913043, 'bought': 0.043478260869565216, 'characters': 0.043478260869565216, 'fictional': 0.043478260869565216, 'has': 0.043478260869565216, 'how': 0.043478260869565216, 'i': 0.06521739130434782, 'is': 0.06521739130434782, 'like': 0.043478260869565216, 'more': 0.043478260869565216, 'my': 0.043478260869565216, 'novel': 0.043478260869565216, 'novels': 0.043478260869565216, 'one': 0.043478260869565216, 'than': 0.043478260869565216, 'the': 0.043478260869565216, 'they': 0.043478260869565216, 'this': 0.08695652173913043}
------------------------------------------- 

Probability of words for "question" class 

{'are': 0.05128205128205128, 'author': 0.05128205128205128, 'book': 0.10256410256410256, 'characters': 0.05128205128205128, 'favorite': 0.05128205128205128, 'have': 0.05128205128205128, 'is': 0.10256410256410256, 'my': 0.05128205128205128, 'novels': 0.0512820512

In [21]:
# Calculate prior for one class

def prior(className):    
    denominator = len(stmt_docs) + len(question_docs)
    
    if className == 'statement':
        numerator =  len(stmt_docs)
    else:
        numerator =  len(question_docs)
        
    return np.divide(numerator,denominator)
    
# Calculate class conditional probability for a sentence
    
def classCondProb(sentence, className):
    words = get_words(sentence)
    prob = 1
    for word in words:
        if className == 'statement':
            idx = np.where(word_list_s == word)
            prob = prob * prob_s[np.array(idx)[0,0]]
        else:
            idx = np.where(word_list_q == word)
            prob = prob * prob_q[np.array(idx)[0,0]]   
    
    return prob

# Predict class of a sentence

def predict(sentence):
    prob_statement = classCondProb(sentence, 'statement') * prior('statement')
    prob_question = classCondProb(sentence, 'question') * prior('question')
    if  prob_statement > prob_question:
        return 'statement'
    else:
        return 'question'

### In-lab exercise: Laplace smoothing

Run the code below and figure out why it fails.

When a word does not appear with a specific class in the training data, its class-conditional probability is 0, and we are unable to
get a reasonable probability for that class.

Research Laplace smoothing, and modify the code above to implement Laplace smoothing (setting the frequency of all words with frequency 0 to a frequency of 1).
Run the modified code on the test set.

In [22]:
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])
print('Getting prediction for "%s"' % test_docs[0])
predict(test_docs[0])


Getting prediction for "this is the book"


IndexError: index 0 is out of bounds for axis 1 with size 0

### Exercise 1.1 (10 points)

Explain Why it failed and explain how to solve the problem.

Explanation here! (Double click to explain)

### Reason for failure
- The function get_words(text) expects a list of sentences as its input. If we pass a single string as the input since a string is literally a list of characters  get_words function treats each character of the string input as a word.
- Then subsequently we try to find the class conditional probability of each character using the classCondProb() function which results in an error.

### Exercise 1.2 (20 points)

Modify your code and make it works.

### Corrected code

In [23]:
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])
print('Getting prediction for "%s"' % test_docs[0])
predict(list([test_docs[0]]))

Getting prediction for "this is the book"


'question'

### Implementing Laplace Smoothing

In [24]:
# YOUR CODE HERE

# Find words in question class, bu not in statement class
not_in_statement = np.setdiff1d(word_list_q, word_list_s)

# Find words in statement class, bu not in question class
not_in_question = np.setdiff1d(word_list_s, word_list_q)

# Increace the word frequencies in statement documents by 1
freq_s_laplace = None
freq_s_laplace = freq_s.copy()
for word in freq_s_laplace:
    freq_s_laplace[word] += 1

# Add words in question class, bu not in statement class to freq_s_laplace with a frequency of 1
for word in not_in_statement:
    freq_s_laplace[word] = 1
    
# Increace the word frequencies in question documents by 1
freq_q_laplace = None
freq_q_laplace = freq_q.copy()
for word in freq_q_laplace:
    freq_q_laplace[word] += 1

# Add words in statement class, bu not in question class to freq_q_laplace with a frequency of 1
for word in not_in_question:
    freq_q_laplace[word] = 1
    
# Calculate updated word probabilities  - statement class
freq_s_values = list(freq_s_laplace.values())
sum_freq_s_values = sum(freq_s_values)
prob_s_laplace = {}
for word in freq_s_laplace:
    prob_s_laplace[word] = float(freq_s_laplace[word])/ sum_freq_s_values

# Calculate updated word probabilities  - question class
freq_q_values = list(freq_q_laplace.values())
sum_freq_q_values = sum(freq_q_values)
prob_q_laplace = {}
for word in freq_q_laplace:
    prob_q_laplace[word] = float(freq_q_laplace[word])/ sum_freq_q_values

In [25]:
# Calculate class conditional probability for a sentence with Laplace smoothing
def classCondProb_Laplace(sentence, className):
    words = get_words(sentence)
    prob = 1
    for word in words:
        if className == 'statement':
            if word in prob_s_laplace:
                prob = prob * prob_s_laplace[word]
        else:
            if word in prob_q_laplace:
                prob = prob * prob_q_laplace[word] 
    
    return prob

# Predict class of a sentence with Laplace smoothing
def predict(sentence):
    prob_statement = classCondProb_Laplace(sentence, 'statement') * prior('statement')
    prob_question = classCondProb_Laplace(sentence, 'question') * prior('question')
    if  prob_statement > prob_question:
        return 'statement'
    else:
        return 'question'


In [26]:
# Test function: Do not remove
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])

for sentence in test_docs:
    print('Getting prediction for %s"' % sentence)
    print(predict(sentence))
    
print("success!")
# End Test function

Getting prediction for this is the book"
statement
Getting prediction for who are the novels characters"
question
Getting prediction for is this the author"
statement
Getting prediction for I like apples"
statement
success!


In [27]:
# Test function: Do not remove
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])

for sentence in test_docs:
    print('Getting prediction for "%s"' % sentence)
    print(predict(list([sentence])))
    
print("success!")
# End Test function

Getting prediction for "this is the book"
question
Getting prediction for "who are the novels characters"
question
Getting prediction for "is this the author"
question
Getting prediction for "I like apples"
statement
success!


**Expect result**:\
Getting prediction for this is the book"\
question\
Getting prediction for who are the novels characters"\
question\
Getting prediction for is this the author"\
question

### Take home exercise

Find a more substantial text classification dataset, clean up the documents, and build your NB classifier. Write a brief report on your in-lab and take home exercises and results.

### Answer

- In this exercise I used the 'apple_twitter_sentiment_texts' dataset from Kaggle available at https://www.kaggle.com/datasets/seriousran/appletwittersentimenttexts.

- This dataset look into the sentiment around Apple, based on tweets containing #AAPL, @apple, etc. Contributors were given a tweet and asked whether the user was positive(1), negative(-1), or neutral(0) about Apple.

### Implementation

In [28]:
import numpy as np
import pandas as pd
import re
import operator
import math
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/anjana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# A function to clean the dataset
def data_clean(X):
    data = X.copy()
    char = '@'
    data['text'] = data['text'].str.lower()
    pattern1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[a-zA-Z]+|$[a-zA-Z]+|@[a-zA-Z]+|[,.^_#$*%-;鶯椰~!?:]')
    pattern2 = re.compile('rt')
    for i in range(len(data['text'])):
        data['text'][i] = " ".join(word for word in data['text'][i].split(' ') if not word.startswith(char))
        data['text'][i] = pattern1.sub('', data['text'][i])
        data['text'][i] = pattern2.sub('', data['text'][i])
        data['text'][i] = re.sub(r'[^A-Za-z ]+', '', data['text'][i])
        
    stop = stopwords.words('english')
    data['text'] = data['text'].str.split()
    data['text'] = data['text'].apply(lambda x: [item for item in x if item not in stop])
    
    for i in range(len(data)):
        data.iloc[i] = " ".join(" ".join(l) for l in data.iloc[i])
    return data

In [30]:
class NaiveBayesTextClassifier(object):
    
    def __init__(self):
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()
        self.y_classes = {}
    
    def _tokenize(self, text):
        return re.split("\W+", text)
    
    def _get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts
    
    def fit(self, X, y):
        # Reset attributes
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()
        self.y_classes = {}
        n = len(X)
        
        self.y_classes = pd.Series(y).value_counts().to_dict()
        for y_class in self.y_classes:
            self.num_messages[y_class] = sum(1 for label in y if label == y_class)
            self.log_class_priors[y_class] = math.log(self.num_messages[y_class] / n)
            self.word_counts[y_class] = {}
        
        for x, y_val in zip(X, y):
            if x == "": continue
            c = y_val
            counts = self._get_word_counts(self._tokenize(x))
            for word, count in counts.items():
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0
                self.word_counts[c][word] += count
                
    def predict(self, X):
        result = []
        for x in X:
            class_score = {}
            for y_class in self.y_classes:
                class_score[y_class] = 0
            if len(x.strip()) != 0:                
                counts = self._get_word_counts(self._tokenize(x))
                for word, _ in counts.items():
                    if word not in self.vocab: continue                
                    for y_class in self.y_classes:
                        # add Laplace smoothing
                        log_w_given_class = math.log( (self.word_counts[y_class].get(word, 0.0) + 1) / (self.num_messages[y_class] + len(self.vocab)) )
                        class_score[y_class] += log_w_given_class 
            for y_class in self.y_classes:
                class_score[y_class] += self.log_class_priors[y_class]
            
            result.append(max(class_score.items(), key=operator.itemgetter(1))[0])
        np_result = np.array(result)
        return np_result
    

In [31]:
data = pd.read_csv("apple-twitter-sentiment-texts.csv")
data.head()

,text,sentiment
0,Wow. Yall needa step it up @Apple RT @heynyla:...,-1
1,What Happened To Apple Inc? http://t.co/FJEX...,0
2,Thank u @apple I can now compile all of the pi...,1
3,The oddly uplifting story of the Apple co-foun...,0
4,@apple can i exchange my iphone for a differen...,0


In [32]:
data["sentiment"].value_counts()

 0    801
-1    686
 1    143
Name: sentiment, dtype: int64

In [33]:
data.isnull().sum()

text         0
sentiment    0
dtype: int64

### Experiment 1 - Using the entire dataset (Classes are unbalanced)

In [34]:
# Setting X and y
X = data[['text']]

y = data['sentiment']

# Train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1141, 1) (489, 1) (1141,) (489,)


In [35]:
# Reset and drop indexes
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)


In [36]:
# Cleaning X_train
X_train_cleaned = data_clean(X_train)

# Converting X_train_cleaned and y_train to Numpy arrays
X_train_cleaned_np = (X_train_cleaned.iloc[:,[0]].values).reshape((-1,))
y_train_np = y_train.to_numpy()

In [37]:
# Training NaiveBayesTextClassifier
nb = NaiveBayesTextClassifier()
nb.fit(X_train_cleaned_np, y_train_np)

In [38]:
# Cleaning X_test
X_test_cleaned = data_clean(X_test)

# Converting X_train_cleaned and y_train to Numpy arrays
X_test_cleaned_np = (X_test_cleaned.iloc[:,[0]].values).reshape((-1,))
y_test_np = y_test.to_numpy()

# Predictions for X_test
y_predict = nb.predict(X_test_cleaned_np)

In [39]:
def getNaiveBayesTextClassifierAccuracy(y, y_predict):
    acc = (y_predict == y).astype(int).sum() / y.shape[0]
    return acc

test_acc = getNaiveBayesTextClassifierAccuracy(y_test_np, y_predict)
print("NaiveBayesTextClassifierAccuracy prediction accuracy - unbalanced dataset: ", test_acc)

NaiveBayesTextClassifierAccuracy prediction accuracy - unbalanced dataset:  0.8077709611451943


### Experiment 2 - Using a balanced dataset

In [40]:
# 143 samples with 'positive' sentiment class
positive_tweets = data.loc[data['sentiment'] == 1]
print(positive_tweets.shape)
positive_tweets.head(3)

(143, 2)


,text,sentiment
2,Thank u @apple I can now compile all of the pi...,1
33,Finally! Brooklyn Is Getting Its Own #Apple St...,1
34,RT @SwiftKey: We're so excited to be named to ...,1


In [41]:
# Selecting 143 samples at random form 'negative' sentiment class
negative_tweets = data.loc[data['sentiment'] == -1].sample(n=143, random_state=999)
print(negative_tweets.shape)
negative_tweets.head(3)

(143, 2)


,text,sentiment
757,@Apple yo this Genius Bar shit is worst than t...,-1
1544,Hey @apple how about you guys make a charger t...,-1
707,All of my apple product chargers look like tha...,-1


In [42]:
# Selecting 143 samples at random form 'neutral' sentiment class
neutral_tweets = data.loc[data['sentiment'] == 0].sample(n=143, random_state=999)
print(neutral_tweets.shape)
neutral_tweets.head(3)

(143, 2)


,text,sentiment
63,Is Apple The Most Important Stock On Earth? h...,0
944,Steve Wozniak calls @Apple's legendary garage ...,0
1551,5 Companies Growing Faster Than Apple Inc. htt...,0


In [43]:
data_balanced = pd.concat([positive_tweets, negative_tweets, neutral_tweets])
data_balanced['sentiment'].value_counts()

 1    143
-1    143
 0    143
Name: sentiment, dtype: int64

In [44]:
# Setting X and y
X_b = data_balanced[['text']]

y_b = data_balanced['sentiment']

# Train, test split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size=0.3, random_state=999)

print(X_train_b.shape, X_test_b.shape, y_train_b.shape, y_test_b.shape)

(300, 1) (129, 1) (300,) (129,)


In [45]:
# Reset and drop indexes
X_train_b.reset_index(drop=True, inplace=True)
X_test_b.reset_index(drop=True, inplace=True)

In [46]:
# Cleaning X_train
X_train_cleaned_b = data_clean(X_train_b)

# Converting X_train_cleaned and y_train to Numpy arrays
X_train_cleaned_np_b = (X_train_cleaned_b.iloc[:,[0]].values).reshape((-1,))
y_train_np_b = y_train_b.to_numpy()

In [47]:
# Training NaiveBayesTextClassifier
nb_b = NaiveBayesTextClassifier()
nb_b.fit(X_train_cleaned_np_b, y_train_np_b)

In [48]:
# Cleaning X_test
X_test_cleaned_b = data_clean(X_test_b)

# Converting X_train_cleaned and y_train to Numpy arrays
X_test_cleaned_np_b = (X_test_cleaned_b.iloc[:,[0]].values).reshape((-1,))
y_test_np_b = y_test_b.to_numpy()

# Predictions for X_test
y_predict_b = nb_b.predict(X_test_cleaned_np_b)

In [49]:
test_acc_b = getNaiveBayesTextClassifierAccuracy(y_test_np_b, y_predict_b)
print("NaiveBayesTextClassifierAccuracy prediction accuracy - balanced dataset: ", test_acc_b)

NaiveBayesTextClassifierAccuracy prediction accuracy - balanced dataset:  0.6976744186046512


### Discussion

- Following are the steps I followed in this exercise to implement a Naive Bayes Classifier for text classification

**Load and analyze the dataset**
- Once the CSV file is loaded and a basic analysis was done to check for null values etc. The dataset has two columns out of which X is set to column `text` and y is set to column `sentiment`, since the target of this exercise is to predict sentiment of a given tweet(text).
- When I analyzed the dataset it was clear that the dataset is unbalanced.

| Class  |  No. of samples |
| :----: | --- |
| Positive Sentiment (1) | 143 |
|  Neutral Sentiment (0)| 801 |
| Negative Sentiment (-1) | 686 |

- In the first training (Experiment 1) I used the entire dataset and in the 2nd training (Experiment 2) I used a balanced dataset by selecting 143 samples from each class.
- In both experiments steps I followed are described below.


**Train test split**
- 70% of the samples were used in training and 30% were used in testing accuracy of the model

**Cleaning the X dataset**
- The function `data_clean()` above clean `text` in several steps.
    - Since tweets contain a lot of Twitter handles starting with '@' all such words were removed.
    - Next, all words were converted to lowercase.
    - URLs starting with http, words starting with \'#\', \'$\' were removed.
    - Since tweets contain the phrase 'RT' to indicate re-tweets phrase 'RT' was removed.
    - Commonly used stopwords like ‘and’, ‘the’, and ‘at’ that do not hold any special meaning in a sentence are also removed using Python Natural Language Toolkit(nltk)'s English stopwords list.
    - Basically, the dataset is cleaned such that only words and space are remaining.

**Implement `NaiveBayesTextClassifier` class**
- The frequency of some of the words was extremely low. Thus the probability values were very small. Multiplying many small probabilities together resulted in values that rounded down to zero, causing runtime errors. To fix this both in fit and predict functions log-likelihoods were computed instead of likelihoods
- Also to deal with words that do not appear in a specific class of the training data, thus causing a class-conditional probability of 0, Laplace smoothing was used.

**Model training and prediction**
- Using `data_clean()` function, tweets in training dataset were cleaned. The model was trained using the cleaned dataset.
- Then tweets in X_test were also cleaned using `data_clean()` function and predicted values for `sentiment` were obtained for cleaned X_test.

**Model accuracy**
- Model accuracy comparison:

| Experiment  |  Accuracy |
| :----: | --- |
| Model Trained using entire dataset | 0.8077709611451943 |
| Model Trained using a balanced dataset | 0.6976744186046512 |

- Even though the `NaiveBayesTextClassifier` I developed results in an accuracy of 0.8077709611451943 when trained using the entire dataset, its accuracy drops to 0.6976744186046512 when the training is done using a balanced dataset.